In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def next_month(ym: str) -> str:
    return (pd.Period(ym, freq="M") + 1).strftime("%Y-%m")

def get_att_data(emCode, cycle, Authorization):
    url = "https://im.360teams.com/api/qfin-api/securityapi/attendance/query/detail"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "AppKey": "360teams",
        "Authorization": Authorization,
    }
    resp = requests.get(url, headers=headers, params={"emCode": emCode, "attDate": "", "cycle": cycle})
    data = resp.json()
    if resp.ok and data.get('code') == 0:
        print(f"{cycle} 请求成功！")
        return data
    print(f"{cycle} 请求失败！{data if resp.ok else resp.status_code}")

def parsed_att_date(check_month, emCode, Authorization, verbose=True):
    # 获取数据
    data = []
    for m in [check_month, next_month(check_month)]:
        data += get_att_data(emCode, m, Authorization)['data']['calendarList']
    
    # 构建DataFrame
    att_df = pd.DataFrame([{
        '日期': d['attDate'], '月份': d['attDate'][:7], '假期flag': d['isrest'],
        '上班打卡': d['firstDate'], '下班打卡': d['endDate'], 
        '备注': d['exp'], '备注2': d['resultList'][0] if d['resultList'] else None
    } for d in data])
    
    att_df = att_df[att_df['月份'] == check_month].drop_duplicates().reset_index(drop=True)
    att_df['工时'] = (pd.to_datetime(att_df['下班打卡']) - pd.to_datetime(att_df['上班打卡'])).dt.total_seconds() / 3600
    att_df.loc[att_df['假期flag'] != 0, '工时'] = 0
    
    # 计算有效工作日
    def calc_workday(s):
        if s['假期flag'] != 0: return 0
        if pd.isna(s['备注']) or s['备注'] in ['迟到', '早退']: return 1
        if s['备注'] in ['病假', '年假', '调休假']: return 0.5 if s['工时'] > 0 else 0
        return 0.5 if s['备注2'] and '半天' in s['备注2'] else 0
    
    att_df['有效工作日'] = att_df.apply(calc_workday, axis=1)
    
    # 统计
    today = pd.Timestamp.now().strftime("%Y-%m-%d")
    calc_data = att_df[att_df['日期'] < today]
    valid_days, valid_hours = calc_data['有效工作日'].sum(), calc_data['工时'].sum()
    avg_hours = round(valid_hours / valid_days, 2)
    
    if verbose:
        print(f"\n当前日期 {today}\n检查月份 {check_month}\n有效工作日 {valid_days}\n有效工时 {round(valid_hours,2)}\n平均工时 {avg_hours}")
    
    return [avg_hours, att_df]


In [8]:
att_detailed = parsed_att_date(
    check_month='2026-01'
    ,emCode='JR1001913'
    ,Authorization='97611d5d17f4447da7e806bb9697b64a'
    ,verbose=True
)

2026-01 请求成功！
2026-02 请求成功！

当前日期 2026-01-14
检查月份 2026-01
有效工作日 7.5
有效工时 79.35
平均工时 10.58
